C++ presentation of 'Templet application' interface and model
=============================================================

```
 Роли и команды
 ==============

 ----------------------------------------------------------------------
             Commands           |                Roles
 ----------------------------------------------------------------------
  REST commands |  CLI commands | RESOURCE | OBSERVER | CLEANER | USER 
 ---------------|---------------|--------------------------------------
 write_event    |  event        |    +     |          |         |
 read_event     |  log          |    +     |    +     |         |
 reply_on_query |  reply        |    +     |          |         |
 write_query    |  query        |          |          |         |   +
 read_answer    |  query        |          |    +     |         |   +
 upload_blob    |  upload       |    +     |          |         |   +
 download_blob  |  download     |    +     |    +     |         |   +
 delete_blob    |  erase        |          |          |    +    |

```

In [3]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // resource and user REST interface    

typedef std::string NAME;// to whom the token was issued
typedef long        TOKEN;
typedef long        ORDINAL;
typedef long        TAG;
typedef long        ID;
    
struct SESSION{
    bool is_active = false;
    NAME name;       // who is the session owner?
    bool as_resource;// what access does the session owner have?
    bool as_observer;
    bool as_cleaner;
    bool as_user;
}; 

struct DATA{ long size; void* data; }; //abstract binary data

struct EVENT{
    ORDINAL  ord;
    TAG      tag;
    DATA     data;
    bool     is_query;
    NAME     writer_name;
};

struct ANSWER{
    ORDINAL ord;
    NAME    name;
    DATA    data;
};

struct BLOB{
    ID      id;
    ORDINAL ord_after;
    DATA    data;
    NAME    data_name;  
    NAME    writer_name;
};

/*-1-*/  
bool open_session(/*out*/SESSION&s,
         /*authentication info*/NAME&name,
         /*authorization info*/ bool is_active, bool as_resource, bool as_observer, bool as_cleaner, bool as_user){   
    s.name = name;

    s.is_active = is_active;
    s.as_resource = as_resource;
    s.as_observer = as_observer;
    s.as_cleaner = as_cleaner;
    s.as_user = as_user;
   
    return s.is_active;
}
    
/*-2-*/    
bool close_session(/*in/out*/SESSION& s){
    if(s.is_active){s.is_active=false;return true;}
    else return false;
}

/*-3-*/         
bool write_event(/*in*/SESSION&s,/*in*/TAG t,/*in*/DATA d,/*out*/ORDINAL&ord){
    if(!(s.is_active && s.as_resource))return false;
    
    EVENT ev;
    
    ev.ord = event_table.size()+1; 
    ev.tag = t;
    ev.data = d;
    ev.is_query = false;
    ev.writer_name = s.name;
    
    event_table.push_back(ev);
    event_by_ord_indx[ev.ord] = &(event_table.back());
    
    return true;
}
    
/*-4-*/              
bool read_event(/*in*/SESSION&s, /*in*/ORDINAL ord,/*out*/EVENT&ev){
    if(!(s.is_active && (s.as_resource || s.as_observer)))return false;
    
    try{
        ev = *event_by_ord_indx.at(ord);
        return true;
    }
    catch(...){
        return false;
    }
}
    
/*-5-*/    
bool reply_on_query(/*in*/SESSION&s,/*in*/ORDINAL ord,/*in*/DATA d){
    if(!(s.is_active && s.as_resource))return false;
    
    try{
        ANSWER answ;
        
        answ.ord  = ord; 
        answ.name = event_by_ord_indx.at(ord)->writer_name;
        answ.data = d;
    
        answer_table.push_back(answ);
        answer_by_ord_indx[ord] = &(answer_table.back());
    
        return true;
    }
    catch(...){
        return false;
    }
}
    
/*-6-*/
bool write_query(/*in*/SESSION&s,/*in*/TAG t,/*in*/DATA d,/*out*/ORDINAL&){
    if(!(s.is_active && s.as_user))return false;
    
    EVENT ev;
    
    ev.ord = event_table.size()+1; 
    ev.tag = t;
    ev.data = d;
    ev.is_query = true;
    ev.writer_name = s.name;
    
    event_table.push_back(ev);
    event_by_ord_indx[ev.ord] = &(event_table.back());
    
    return true;
}
    
/*-7-*/            
bool read_answer(/*in*/SESSION&s,/*in*/ORDINAL ord,/*out*/DATA&d){
    if(!(s.is_active && (s.as_user || s.as_observer)))return false;
    
    try{
        ANSWER* answ = answer_by_ord_indx.at(ord);
        if(s.name != answ->name || !s.as_observer) return false;
        d = answ->data;
        return true;
    }
    catch(...){
        return false;
    }
}
    
/*-8-*/   
bool upload_blob(/*in*/SESSION&s,/*in*/DATA d,/*in*/NAME name,/*out*/ID& id){
    if(!(s.is_active && (s.as_user || s.as_resource)))return false;
    
    id = rand(); //create or calculate ID
    
    BLOB blob;
    
    blob.id = id;
    blob.ord_after = event_table.size();
    blob.data = d;
    blob.data_name = name;
    blob.writer_name = s.name;
    
    blob_table.push_back(blob);
    blob_by_id_indx[id] = &(blob_table.back());
    
    return true;
}
    
/*-9-*/    
bool download_blob(/*in*/SESSION&s,/*in*/ID id,/*out*/BLOB& blob){
    if(!(s.is_active && (s.as_user || s.as_resource || s.as_observer)))return false;
      
    try{
        blob = *blob_by_id_indx.at(id);
        return true;
    }
    catch(...){
        return false;
    }
}
    
/*-10-*/    
bool delete_blob(/*in*/SESSION&s,/*in*/ID id){
    if(!(s.is_active && s.as_cleaner))return false;
           
    try{
        blob_by_id_indx.erase(id);
        for(auto it = blob_table.begin(); it!=blob_table.end();it++){
            if(it->id == id){
                blob_table.erase(it); 
                return true;
            }
        }        
    }
    catch(...){
        return false;
    }
    
    return true;
}

private: // admin interface, admin has direct access to app state
    
list<EVENT>         event_table;
map<ORDINAL,EVENT*> event_by_ord_indx; 

list<ANSWER>        answer_table;
map<ORDINAL,ANSWER*>answer_by_ord_indx;

list<BLOB>          blob_table;
map<ID,BLOB*>       blob_by_id_indx;

};

```
Интерфейс параметров командной строки
=====================================

Идея: возможно реализовать сервис синхронизации глобального состояния на базе готовых систем удаленного запуска задач (Everest, HTCondor, Dirac и т.д.) для быстрого прототипирования и выполнения сравнительного нагрузочного тестирования предлагаемой архитектуры приложения. Для этого реализуется консольное приложение-"псевдозадача" с описанным ниже интерфейсом.

Параметры
=========
(--base | -b) <base> база данных состояния приложения
(--tag  | -t) <num> тэг события
(--ordinal | -о) <num> порядковый номер события
(--answer  | -a) <file> ответ на запрос 

Форматы файлов и stdout
=======================
<file>     формат определяется в приложении
<base>     SQLite файл базы данных, хранящей текущее состояние приложения
<logfile>  запись фрагмента из последовательных событий, внутренный формат
<upload>   список файлов, разделенных символом перевода строки
<download> список пар вида <ключ>/<имя файла>, разделенных символом перевода строки
<erase>    список ключей, разделенных символом перевода строки

Команды
=======
(--event | -e) <file>  запись события, параметры: base, tag, в stdout - порядковый номер события
(--log   | -l) <logfile> выборка из лога событий, начиная с заданного ordinal, параметры: base, ordinal
(--reply | -r) <file>  ответ на запрос с указанным номером ordinal, параметры: base, ordinal
(--erase | -e) <erase> удаление файлов (блоков данных) из базы состояния приложения, параметры: base
(--query | -q) <file>  файл с данными запроса, параметры: base, tag, answer (если указан, выполняется ожидание ответа)
(--upload   | -u) <upload> загрузка файлов в базу состояния приложения, параметры: base, в stdout - ключи в формате <download>
(--download | -d) <download> выгрузка файлов из базы состояния приложения, параметры: base в stdout - файлы в формате <upload>
```